# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [1]:
import sys
import os
import time
import networkx as nx

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [2]:
import multiprocessing

NUM_PROCESSES = multiprocessing.cpu_count()
print("Number of cpu : ", NUM_PROCESSES)

Number of cpu :  12


In [3]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.instances.reader import SteinlibReader, DatReader

from pcstp.steinertree import SteinerTreeProblem
from pcstp.solver.base import computes_steiner_cost
from pcstp.solver.aco import AntColony
from pcstp.solver.greedy_h1 import GreedyH1

from pcstp.utils.graph import preprocessing
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [4]:
SEED = 100

## Solution obtained with Ant Colony Optimization

In [8]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/RPCST-cologne/cologne1'
NUM_EXPERIMENTS_PER_INSTANCE = 5

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

aco_history = []
solutions = {}
for filename in files:
    if filename.endswith('.xlsx') or filename.endswith('.csv'): continue
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)

    def run_experiment(experiment: int):
        aco_params = dict(
            iterations=50,
            num_ants=len(terminals),
            evaporation_rate=0.5,
            alpha=1.0,
            beta=3.0,
            # beta_evaporation_rate=0.2,
            initial_pheromone=0.1,
            pheromone_amount=2.0,
            pheromone_deposit_strategy='traditional',
            pheromone_initialization_strategy='same_value',
            choose_best=0.2,
            log_level='info',
            early_stopping=10,
            normalize_distance_prize=False,
            allow_edge_perturbation=False,
            ant_max_moves=1000,
            seed=SEED * experiment
        )
        solver = AntColony(
            graph=stp_preprocessed.graph,
            terminals=stp_preprocessed.terminals,
            **aco_params
        )
        steiner_tree, steiner_cost = solver.solve()

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": steiner_cost,
            "duration": solver._duration
        }
        history.update(aco_params)
        return history, solver

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    aco_history.extend([result[0] for result in experiments_results])
    solutions[filename] = [result[1] for result in experiments_results]

Reading: ../data/instances/benchmark/RPCST-cologne/cologne1/i102M2.stp


2022-02-06 10:09:40,420 - [aco.py:295] - MainThread - INFO - Early Stopping: 11
2022-02-06 10:09:40,423 - [aco.py:302] - MainThread - INFO - Best Iteration: 0 - Best Cost: 361269.43657
2022-02-06 10:09:50,735 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:09:50,738 - [aco.py:302] - MainThread - INFO - Best Iteration: 1 - Best Cost: 370316.743089
2022-02-06 10:10:01,099 - [aco.py:295] - MainThread - INFO - Early Stopping: 13
2022-02-06 10:10:01,102 - [aco.py:302] - MainThread - INFO - Best Iteration: 6 - Best Cost: 380730.842958
2022-02-06 10:10:13,666 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-06 10:10:13,668 - [aco.py:302] - MainThread - INFO - Best Iteration: 8 - Best Cost: 385556.56758499995
2022-02-06 10:10:21,096 - [aco.py:295] - MainThread - INFO - Early Stopping: 15
2022-02-06 10:10:21,098 - [aco.py:302] - MainThread - INFO - Best Iteration: 8 - Best Cost: 376204.03593300004


Reading: ../data/instances/benchmark/RPCST-cologne/cologne1/i101M2.stp


2022-02-06 10:12:42,486 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:12:42,490 - [aco.py:302] - MainThread - INFO - Best Iteration: 2 - Best Cost: 392468.52765000006
2022-02-06 10:12:43,377 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:12:43,381 - [aco.py:302] - MainThread - INFO - Best Iteration: 11 - Best Cost: 401038.662651
2022-02-06 10:12:51,436 - [aco.py:295] - MainThread - INFO - Early Stopping: 13
2022-02-06 10:12:51,439 - [aco.py:302] - MainThread - INFO - Best Iteration: 6 - Best Cost: 399101.98106500006
2022-02-06 10:12:53,054 - [aco.py:295] - MainThread - INFO - Early Stopping: 13
2022-02-06 10:12:53,058 - [aco.py:302] - MainThread - INFO - Best Iteration: 11 - Best Cost: 389151.339269
2022-02-06 10:12:58,367 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-06 10:12:58,370 - [aco.py:302] - MainThread - INFO - Best Iteration: 5 - Best Cost: 394891.22670600004


Reading: ../data/instances/benchmark/RPCST-cologne/cologne1/i103M2.stp


2022-02-06 10:16:44,728 - [aco.py:295] - MainThread - INFO - Early Stopping: 13
2022-02-06 10:16:44,731 - [aco.py:302] - MainThread - INFO - Best Iteration: 3 - Best Cost: 549463.410431
2022-02-06 10:16:59,036 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-06 10:16:59,038 - [aco.py:302] - MainThread - INFO - Best Iteration: 3 - Best Cost: 526378.087238
2022-02-06 10:16:59,968 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-06 10:16:59,971 - [aco.py:302] - MainThread - INFO - Best Iteration: 12 - Best Cost: 512077.205737
2022-02-06 10:17:00,476 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-06 10:17:00,478 - [aco.py:302] - MainThread - INFO - Best Iteration: 12 - Best Cost: 517330.49211500003
2022-02-06 10:17:12,280 - [aco.py:295] - MainThread - INFO - Early Stopping: 15
2022-02-06 10:17:12,283 - [aco.py:302] - MainThread - INFO - Best Iteration: 8 - Best Cost: 532834.348844


Reading: ../data/instances/benchmark/RPCST-cologne/cologne1/i104M2.stp


2022-02-06 10:17:59,830 - [aco.py:295] - MainThread - INFO - Early Stopping: 11
2022-02-06 10:17:59,833 - [aco.py:302] - MainThread - INFO - Best Iteration: 0 - Best Cost: 85617.062013
2022-02-06 10:18:00,794 - [aco.py:295] - MainThread - INFO - Early Stopping: 11
2022-02-06 10:18:00,798 - [aco.py:302] - MainThread - INFO - Best Iteration: 0 - Best Cost: 66724.167027
2022-02-06 10:18:01,112 - [aco.py:295] - MainThread - INFO - Early Stopping: 11
2022-02-06 10:18:01,115 - [aco.py:302] - MainThread - INFO - Best Iteration: 0 - Best Cost: 85617.062013
2022-02-06 10:18:04,865 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:18:04,867 - [aco.py:302] - MainThread - INFO - Best Iteration: 3 - Best Cost: 66724.167027
2022-02-06 10:18:05,992 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:18:05,994 - [aco.py:302] - MainThread - INFO - Best Iteration: 1 - Best Cost: 66724.167027


Reading: ../data/instances/benchmark/RPCST-cologne/cologne1/i105M2.stp


2022-02-06 10:18:57,624 - [aco.py:295] - MainThread - INFO - Early Stopping: 11
2022-02-06 10:18:57,627 - [aco.py:302] - MainThread - INFO - Best Iteration: 0 - Best Cost: 66788.171472
2022-02-06 10:19:01,904 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:19:01,907 - [aco.py:302] - MainThread - INFO - Best Iteration: 3 - Best Cost: 66788.171472
2022-02-06 10:19:02,194 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:19:02,197 - [aco.py:302] - MainThread - INFO - Best Iteration: 3 - Best Cost: 66788.171472
2022-02-06 10:19:03,007 - [aco.py:295] - MainThread - INFO - Early Stopping: 12
2022-02-06 10:19:03,010 - [aco.py:302] - MainThread - INFO - Best Iteration: 1 - Best Cost: 66788.171472
2022-02-06 10:19:03,992 - [aco.py:295] - MainThread - INFO - Early Stopping: 13
2022-02-06 10:19:03,995 - [aco.py:302] - MainThread - INFO - Best Iteration: 5 - Best Cost: 66788.171472


In [23]:
experiment = 0
G = solutions['../data/instances/benchmark/RPCST-cologne/cologne1/i104M2.stp'][experiment].graph
steiner_tree = solutions['../data/instances/benchmark/RPCST-cologne/cologne1/i104M2.stp'][experiment].steiner_tree

In [ ]:
sum(list(nx.get_node_attributes(G, 'prize').values()))

In [21]:
steiner_tree.nodes

NodeView((738,))

In [ ]:
G.nodes[78]

In [ ]:
import pandas as pd

df_score_aco = pd.DataFrame.from_dict(aco_history)
df_score_aco.to_csv(os.path.join(INSTANCES_PATH_PREFIX, 'ACO.csv'))

In [ ]:
df_score_aco